# Using Pandas and SQL alchemy to Connect to a Database

### Install SQL Alchemy

In [4]:
!pip install sqlalchemy
!conda install -y -q pymssql

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\user\Anaconda3

  added / updated specs:
    - pymssql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.27 |                0         124 KB
    certifi-2019.11.28         |           py37_0         154 KB
    conda-4.8.1                |           py37_0         2.8 MB
    freetds-1.00.97            |       h7a1dbc1_0         711 KB
    pymssql-2.1.4              |   py37hfa6e2cd_0         166 KB
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

  freetds            pkgs/main/win-64::freetds-1.00.97-h7a1dbc1_0
  pymssql            pkgs/main/win-64::pymssql-2.1.4-py37hfa6e2cd_0

The following packages will be UPDATED:

  ca-certificates         

### Import packages we need

In [1]:
import sqlalchemy as sa
import pandas as pd
from sqlalchemy import create_engine

### Create a connection to our database

### Credentials
##### NB! Change the credentials to those specific to you
##### NB! Make sure you have data in your data base to query from - This train assumes you have a table called football_players in your RDS database

In [2]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = "Andile"

### 1 Create connection

In [5]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

### 2 What is the sql query we want to perform?

In [6]:
sql_query = "Select * from football_players"

### 3 Using pandas we now take our connection and our query to create a dataframe

In [7]:
df = pd.read_sql_query(sql_query, connection)

In [8]:
df.head()

,Name,Age,Nationality,Overall,Acceleration,Aggression,Agility,Balance,Ball_control,Composure,...,Short_passing,Shot_power,Sliding_tackle,Sprint_speed,Stamina,Standing_tackle,Strength,Vision,Volleys,Preferred_Positions
0,Cristiano Ronaldo,32,Portugal,94,89,63,89,63,93,95,...,83,94,23,91,92,31,80,85,88,ST LW
1,L. Messi,30,Argentina,93,92,48,90,95,95,96,...,88,85,26,87,73,28,59,90,85,RW
2,Neymar,25,Brazil,92,94,56,96,82,95,92,...,81,80,33,90,78,24,53,80,83,LW
3,L. Suárez,30,Uruguay,92,88,78,86,60,91,83,...,83,87,38,77,89,45,80,84,88,ST
4,M. Neuer,31,Germany,92,58,29,52,35,48,70,...,55,25,11,61,44,10,83,70,11,GK


### 4 Using Pandas create a view in SQL

In [10]:
class CreateView(Executable, ClauseElement):
    def __init__(self, name, select):
        self.name = name
        self.select = select

@compiles(CreateView)
def visit_create_view(element, compiler, **kw):
    return "CREATE VIEW %s AS %s" % (
         element.name,
         compiler.process(element.select, literal_binds=True)
         )

# import
from sqlalchemy import MetaData, Column, Integer
from sqlalchemy.engine import create_engine
engine = create_engine('sqlite://')
metadata = MetaData(engine)

# create view
createview = CreateView('Top23RSASquadPlayers', football_players.select().where(football_players.Nationality='South Africa'))
engine.execute(createview)

# reflect view and print result
v = Table('Top23RSASquadPlayers', metadata, autoload=True)
for r in engine.execute(v.select()):
    print (r)

SyntaxError: keyword can't be an expression (<ipython-input-10-f83d68a48931>, line 20)